In [311]:
import alpaca
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest, StockQuotesRequest, StockLatestQuoteRequest
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.trading.client import TradingClient
import pandas as pd
import numpy as np
import os, toml
from dotenv import load_dotenv
from time import sleep
from tqdm import tqdm
from alpaca.data.enums import Adjustment

import warnings
warnings.filterwarnings("ignore")

STANDARD_DATETIME_STR = '%Y-%m-%d %H:%M:%S'

load_dotenv(override=True)
livepaper = os.getenv('LIVEPAPER')
config = toml.load('../config.toml')

# Replace with your Alpaca API credentials
API_KEY = config[livepaper]['key']
API_SECRET = config[livepaper]['secret']


In [312]:

# Initialize Alpaca client
client = TradingClient(API_KEY, API_SECRET, paper=True)
data_client = StockHistoricalDataClient(API_KEY, API_SECRET)

# Retrieve all available stocks
assets = client.get_all_assets()
tickers = [asset.symbol for asset in assets if asset.tradable]
print(len(tickers))

# Display the first few tickers
# print("Available Tickers:", tickers[:10])


11292


In [313]:
def calculate_indicators(data):
    # Calculate Average True Range (ATR)
    data['High-Low'] = data['high'] - data['low']
    data['High-Close'] = np.abs(data['high'] - data['close'].shift())
    data['Low-Close'] = np.abs(data['low'] - data['close'].shift())
    data['TR'] = data[['High-Low', 'High-Close', 'Low-Close']].max(axis=1)
    data['ATR'] = data['TR'].rolling(window=14).mean()
    
    # Calculate ADX
    data['+DM'] = np.where((data['high'] - data['high'].shift(1)) > (data['low'].shift(1) - data['low']), data['high'] - data['high'].shift(1), 0)
    data['+DM'] = np.where(data['+DM'] < 0, 0, data['+DM'])
    data['-DM'] = np.where((data['low'].shift(1) - data['low']) > (data['high'] - data['high'].shift(1)), data['low'].shift(1) - data['low'], 0)
    data['-DM'] = np.where(data['-DM'] < 0, 0, data['-DM'])
    data['TR14'] = data['TR'].rolling(window=14).sum()
    data['+DM14'] = data['+DM'].rolling(window=14).sum()
    data['-DM14'] = data['-DM'].rolling(window=14).sum()
    data['+DI14'] = 100 * (data['+DM14'] / data['TR14'])
    data['-DI14'] = 100 * (data['-DM14'] / data['TR14'])
    data['DX'] = (np.abs(data['+DI14'] - data['-DI14']) / (data['+DI14'] + data['-DI14'])) * 100
    data['ADX'] = data['DX'].rolling(window=14).mean()

    # Add temporal features
    data['DayOfWeek'] = data.index.get_level_values('timestamp').dayofweek
    data['HourOfDay'] = data.index.get_level_values('timestamp').hour

In [314]:
def screen_stocks(tickers, start_date, end_date, bar_minutes, atr_threshold=1.0, adx_threshold=25, min_volume=50000):
    screened_stocks = []
    removed_stocks = []
    results = []

    dfs = []
    
    batch_size = 10
    for i in tqdm(range(0, len(tickers), batch_size)):
        batch = tickers[i:i+batch_size]
        try:
            # request_params = StockBarsRequest(
            #     symbol_or_symbols=batch,
            #     timeframe=TimeFrame(amount=bar_minutes, unit=TimeFrameUnit.Minute),  # Corrected TimeFrame
            #     start=start_date,
            #     end=end_date,
            #     adjustment=Adjustment.ALL
            # )
            # bars = data_client.get_stock_bars(request_params)
            
            # request_params = StockQuotesRequest(
            #     symbol_or_symbols=batch,
            #     # timeframe=TimeFrame(amount=bar_minutes, unit=TimeFrameUnit.Minute),  # Corrected TimeFrame
            #     start=start_date,
            #     end=end_date,
            #     limit = 40,
            #     # adjustment=Adjustment.ALL
            # )

            # bars = data_client.get_stock_quotes(request_params)
            
            
            request_params = StockLatestQuoteRequest(
                symbol_or_symbols=batch,
                # timeframe=TimeFrame(amount=bar_minutes, unit=TimeFrameUnit.Minute),  # Corrected TimeFrame
                start=start_date,
                end=end_date,
                # limit = 40,
                # adjustment=Adjustment.ALL
            )

            bars = data_client.get_stock_latest_quote(request_params)
            print(bars)
            
            # df = bars.df
            # # print(df)
            # # Process each stock in the batch
            # for ticker in batch:
            #     try:
            #         stock_data = df[df.index.get_level_values('symbol') == ticker]
            #         calculate_indicators(stock_data)

            #         avg_atr = stock_data['ATR'].mean()
            #         avg_adx = stock_data['ADX'].mean()
            #         avg_volume = stock_data['volume'].mean()
                    
            #         if avg_atr < atr_threshold and avg_adx > adx_threshold and avg_volume > min_volume:
            #             screened_stocks.append(ticker)
            #         else:
            #             removed_stocks.append(ticker)
                    
                    
            #         dfs.append(stock_data[['ATR','ADX','volume','open','high','low','close','volume','trade_count','vwap']].reset_index())
            #         results.append({
            #             'Ticker': ticker,
            #             'Average ATR': avg_atr,
            #             'Average ADX': avg_adx,
            #             'Average Volume': avg_volume
            #         })

            #     except Exception as e:
            #         print(f"Error processing {ticker}: {e}")
            
            sleep(1)  # Sleep to avoid hitting API rate limits
            
        except Exception as e:
            print(f"Error fetching data for batch {batch}: {e}")
    
    # return screened_stocks, removed_stocks, results, pd.concat(dfs)

In [315]:
# Example tickers (should be replaced with the actual list from the asset retrieval)
tickers = [
    "AAPL", "MSFT", "GOOGL", "AMZN", "META", "TSLA", "NVDA", "NFLX", "FB", "BABA",
    "JNJ", "V", "JPM", "PG", "DIS", "MA", "UNH", "HD", "VZ", "ADBE",
    "INTC", "PFE", "KO", "CSCO", "PEP", "MRK", "ABT", "T", "WMT", "XOM",
    "NKE", "MCD"
]

tickers = [
    "AAPL","MSFT"
]


start='2024-01-01'
end='2024-02-01'

# Screen stocks
# start='2022-01-01'
# end='2024-07-08'

while True:
    screen_stocks(tickers, start, end, 1)
    
screened_stocks, removed_stocks, results, all_data = screen_stocks(tickers, start, end, 1)


# all_data.to_csv('all_data.csv',index=False)


print("Screened Stocks: ", len(screened_stocks))
print("Removed Stocks: ", len(removed_stocks))


  0%|          | 0/1 [00:00<?, ?it/s]

{'AAPL': {   'ask_exchange': 'V',
    'ask_price': 228.59,
    'ask_size': 1.0,
    'bid_exchange': 'V',
    'bid_price': 228.56,
    'bid_size': 1.0,
    'conditions': ['R'],
    'symbol': 'AAPL',
    'tape': 'C',
    'timestamp': datetime.datetime(2024, 7, 9, 19, 59, 51, 606332, tzinfo=TzInfo(UTC))}, 'MSFT': {   'ask_exchange': 'V',
    'ask_price': 469.0,
    'ask_size': 2.0,
    'bid_exchange': 'V',
    'bid_price': 459.51,
    'bid_size': 1.0,
    'conditions': ['R'],
    'symbol': 'MSFT',
    'tape': 'C',
    'timestamp': datetime.datetime(2024, 7, 9, 19, 59, 59, 865185, tzinfo=TzInfo(UTC))}}


  0%|          | 0/1 [00:00<?, ?it/s]

{'AAPL': {   'ask_exchange': 'V',
    'ask_price': 228.59,
    'ask_size': 1.0,
    'bid_exchange': 'V',
    'bid_price': 228.56,
    'bid_size': 1.0,
    'conditions': ['R'],
    'symbol': 'AAPL',
    'tape': 'C',
    'timestamp': datetime.datetime(2024, 7, 9, 19, 59, 51, 606332, tzinfo=TzInfo(UTC))}, 'MSFT': {   'ask_exchange': 'V',
    'ask_price': 469.0,
    'ask_size': 2.0,
    'bid_exchange': 'V',
    'bid_price': 459.51,
    'bid_size': 1.0,
    'conditions': ['R'],
    'symbol': 'MSFT',
    'tape': 'C',
    'timestamp': datetime.datetime(2024, 7, 9, 19, 59, 59, 865185, tzinfo=TzInfo(UTC))}}


  0%|          | 0/1 [00:00<?, ?it/s]

{'AAPL': {   'ask_exchange': 'V',
    'ask_price': 228.59,
    'ask_size': 1.0,
    'bid_exchange': 'V',
    'bid_price': 228.56,
    'bid_size': 1.0,
    'conditions': ['R'],
    'symbol': 'AAPL',
    'tape': 'C',
    'timestamp': datetime.datetime(2024, 7, 9, 19, 59, 51, 606332, tzinfo=TzInfo(UTC))}, 'MSFT': {   'ask_exchange': 'V',
    'ask_price': 469.0,
    'ask_size': 2.0,
    'bid_exchange': 'V',
    'bid_price': 459.51,
    'bid_size': 1.0,
    'conditions': ['R'],
    'symbol': 'MSFT',
    'tape': 'C',
    'timestamp': datetime.datetime(2024, 7, 9, 19, 59, 59, 865185, tzinfo=TzInfo(UTC))}}


  0%|          | 0/1 [00:00<?, ?it/s]

{'MSFT': {   'ask_exchange': 'V',
    'ask_price': 469.0,
    'ask_size': 2.0,
    'bid_exchange': 'V',
    'bid_price': 459.51,
    'bid_size': 1.0,
    'conditions': ['R'],
    'symbol': 'MSFT',
    'tape': 'C',
    'timestamp': datetime.datetime(2024, 7, 9, 19, 59, 59, 865185, tzinfo=TzInfo(UTC))}, 'AAPL': {   'ask_exchange': 'V',
    'ask_price': 228.59,
    'ask_size': 1.0,
    'bid_exchange': 'V',
    'bid_price': 228.56,
    'bid_size': 1.0,
    'conditions': ['R'],
    'symbol': 'AAPL',
    'tape': 'C',
    'timestamp': datetime.datetime(2024, 7, 9, 19, 59, 51, 606332, tzinfo=TzInfo(UTC))}}


  0%|          | 0/1 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
all_data

,symbol,timestamp,ATR,ADX,volume,open,high,low,close,volume,trade_count,vwap
0,AAPL,2024-01-02 09:00:00+00:00,NaN,NaN,18933.0,191.02,191.02,189.50,189.70,18933.0,612.0,189.76
1,AAPL,2024-01-02 09:01:00+00:00,NaN,NaN,8562.0,189.61,189.63,189.50,189.50,8562.0,425.0,189.58
2,AAPL,2024-01-02 09:02:00+00:00,NaN,NaN,5355.0,189.40,189.40,189.32,189.32,5355.0,129.0,189.35
3,AAPL,2024-01-02 09:03:00+00:00,NaN,NaN,4405.0,189.37,189.37,189.30,189.37,4405.0,83.0,189.36
4,AAPL,2024-01-02 09:04:00+00:00,NaN,NaN,2225.0,189.40,189.45,189.40,189.40,2225.0,87.0,189.42
...,...,...,...,...,...,...,...,...,...,...,...,...
14118,MSFT,2024-01-31 23:54:00+00:00,0.102143,27.253056,3543.0,396.86,396.86,396.80,396.80,3543.0,24.0,396.81
14119,MSFT,2024-01-31 23:55:00+00:00,0.096429,26.384558,585.0,396.81,396.85,396.81,396.85,585.0,18.0,396.83
14120,MSFT,2024-01-31 23:56:00+00:00,0.093571,24.011788,760.0,396.81,396.81,396.81,396.81,760.0,22.0,396.81
14121,MSFT,2024-01-31 23:57:00+00:00,0.087143,20.561920,287.0,396.82,396.82,396.82,396.82,287.0,17.0,396.77


In [ ]:
screened_stocks[:20]

['AAPL']

In [ ]:



# # Function to create the symbol list file
# def create_symbol_list_file(screened_stocks, results, start_date='', end_date='', updates_months=''):
#     # Create a DataFrame from results
#     results_df = pd.DataFrame(results)

#     # Save the results to a CSV file
#     results_df.to_csv('symbol_list.csv', index=False)
    
#     print("Symbol list file created: symbol_list.csv")

# start='2023-01-01'
# end='2024-07-01'

# # Create symbol list file
# create_symbol_list_file(screened_stocks, results, start_date=start, end_date=end, updates_months='12')
